<a href="https://colab.research.google.com/github/CS-Edwards/games_and_friends/blob/main/Part_1_Weaviate_Client_and_Generative_Search_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Games and Friends 😊🎲

Love board games? In this demo we build a workflow with Weaviate (Python v.4) and OpenAI to query our database suggests games to play and friends to play them with using natural language prompts.

This demo is split into two parts (and two notebooks):

-  *Part 1: Weaviate Client and Generative Search
-  Part 2: Open AI Function Calling

**THIS NOTEBOOK IS PART ONE.**

In this notebook we:
1. Connect to Weaviate Cloud Database client instance
2. Import and review out dataset(s)
3. Create "Game" and "Player" Collections and Schema
4. Populate the Collections by importing data using two methods:
  - Batch: `batch.dynamic()`
  - Insert:  `insert_many()`
5. Execute Generative Search on Collections


In [1]:
!pip install -q -U weaviate-client # Python v.4
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.1 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4

In [2]:
import weaviate
import openai
import pandas as pd
from typing import List, Any


from weaviate.classes.config import Configure, Property, DataType, VectorDistances
from weaviate.util import generate_uuid5
from weaviate.classes.query import MetadataQuery

In [11]:
#@title ▶️ Load Local Utility Functions

def print_response_objects(objects: List[Any]):
    """
    Print function for query generative search and generate responses

    Args:
        objects (List[Any]): List of objects to process.
    """
    for o in objects:
        print(f'Properties: {o.properties}')
        print(f'Distance: {o.metadata.distance}')

### SETUP: Weaviate Cloud and Environment Variables

In this demo we use the free 14-day Weavaite Cloud Database sandbox.

To setup your sandbox:
1. Login or create an account at: [console.weaviate.cloud](https://console.weaviate.cloud)

2. Click "Create Cluster", and provide a "Sandbox name" for your instance and select "Create".

3. In your .env, or "Secrets" copy and save the:
  <br> REST Endpoint URL as "WCS_URL"
  <br> Admin API Key as "WCS_API_Key"


Additional Resources:

- Quickstart- Weaviate Cloud: https://weaviate.io/developers/wcs/quickstart
- Documentation - Weaviate Cloud: https://weaviate.io/developers/wcs

In [4]:

#
# Set environment variables
#

# For Colab "Secrets"
from google.colab import userdata

URL = userdata.get("WCS_URL")
APIKEY = userdata.get("WCS_API_KEY")
OPENAIKEY = userdata.get("OPENAI")


##### Note: Jupyter Environtment Variables

To set environment variables for a Jupyter Notebook use:
```
import os
URL = os.getenv("WCS_URL")
APIKEY = os.getenv("WCS_API_KEY")
```



### CLIENT: Weaviate Cloud Database

In [5]:
#
# Connect to WCS instance
#

with weaviate.connect_to_wcs(
    cluster_url=URL,
    auth_credentials=weaviate.auth.AuthApiKey(APIKEY),
    headers={
        "X-OpenAI-Api-Key": OPENAIKEY
    }
  )as client:
    print(f'Weaviate Client Ready: {client.is_ready()}')
    print(f'Weaviate Client Connected: {client.is_connected()}')

Weaviate Client Ready: True
Weaviate Client Connected: True


### DATASETS: Games and Players

🎲 **Game Data: Kaggle and Wikipedia** <br>
The initial data was sourced from [Kaggle's: Board Game Geek Top 100](https://www.kaggle.com/datasets/medaxone/boardgamegeek-top100) dataset. We then pulled Wikipedia data for each game and added the information to the dataset.

Filename: `board_game_dataset.csv` <br>
To view dataset building notebook: Linked here

😊 **Player Data:**<br>
The data for the player profiles was created manually and is stored within the notebook in JSON format.


In [13]:
#
# Import dataset
#

data = pd.read_csv('data/board_game_dataset.csv')
data.head(5)

In [ ]:
data.info()

In [ ]:
#@title ▶️ Load Player Data: player_profiles_json

player_profiles_json = [
  {
    "name": "Khadijah",
    "age": 25,
    "favoriteGame": "Monopoly",
    "profile": "I enjoy competitive games that involve strategy and negotiation. Monopoly is one of my favorite games because it allows me to showcase my business acumen and outwit my opponents."
  },
  {
    "name": "Max",
    "age": 30,
    "favoriteGame": "Scrabble",
    "profile": "As a lover of words and puzzles, Scrabble is my go-to game. I thrive on the challenge of forming words and maximizing points. I also appreciate games that stimulate my mind and vocabulary."
  },
  {
    "name": "Sinclair",
    "age": 22,
    "favoriteGame": "Risk",
    "profile": "Risk appeals to my adventurous spirit and strategic thinking. I enjoy plotting my conquests and engaging in epic battles with friends. Games that require long-term planning and diplomacy are my forte."
  },
  {
    "name": "Regine",
    "age": 28,
    "favoriteGame": "Catan",
    "profile": "Catan is my go-to game for its blend of strategy and social interaction. I relish the challenge of building settlements and trading resources with fellow players. Cooperative games that foster camaraderie are my favorite."
 }
]


In [ ]:
#
# **LOAD player_profiles.json FILE IN PREVIOUS CELL**
# View player profile.
#


players_data = pd.DataFrame(player_profiles_json)
players_data.head()



In [ ]:
players_data.info()

###  COLLECTIONS AND SCHEMA: 'Game' and 'Player'

Based on the data above, we see that each object in our database is either a 'Game' or a 'Player'. Each collection has a distinct schema that is approprite for the data stored within the collection.

In the 'Game' collection schema we vectorize the `tite`, `subtitle`,`primaryType`, `secondaryType`, and `description` properies using OpenAI's `text-embedding-ada-002` embedding model. We exclude the numerical properties from vectorization.

In the 'Player' collection schema we vectorize the `age`, `name`,`profile`, using OpenAI's `text-embedding-ada-002` embedding model. We exclude the `name` property from vectorization.


**Resources:**
-  Weaviate Collections: [Documentation](https://weaviate.io/developers/weaviate/manage-data/collections)
- Open AI Embedding Models: [Documentation](https://platform.openai.com/docs/guides/embeddings/embedding-models)



In [ ]:
#
# Clear Existing Collections
#
client.connect()
client.collections.delete_all()
client.collections.list_all() #empty


In [ ]:
#
# Create Games Collection
#

client.collections.create(
    "Game",
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="subtitle", data_type=DataType.TEXT),
        Property(name="primaryType", data_type=DataType.TEXT),
        Property(name="secondaryType", data_type=DataType.TEXT),
        Property(name="description", data_type=DataType.TEXT),
        Property(name="minPlayers", data_type=DataType.NUMBER, skip_vectorization=True),
        Property(name="maxPlayers", data_type=DataType.NUMBER, skip_vectorization=True),
        Property(name="bestPlayers", data_type=DataType.NUMBER, skip_vectorization=True),
        Property(name="minPlayTime", data_type=DataType.NUMBER, skip_vectorization=True,),
        Property(name="minAge", data_type=DataType.NUMBER, skip_vectorization=True)
    ],
   vectorizer_config=Configure.Vectorizer.text2vec_openai(),
   vector_index_config=Configure.VectorIndex.hnsw(
       distance_metric= VectorDistances.COSINE
   ),
    generative_config=Configure.Generative.openai()

)

In [ ]:
#
# Player Collection
#

client.collections.create(
    "Player",
    properties=[
        Property(name="name", data_type=DataType.TEXT, skip_vectorization=True),
        Property(name="age", data_type=DataType.NUMBER),
        Property(name="favoriteGame", data_type=DataType.TEXT),
        Property(name="profile", data_type=DataType.TEXT)
    ],
   vectorizer_config=Configure.Vectorizer.text2vec_openai(),
   vector_index_config=Configure.VectorIndex.hnsw(
       distance_metric= VectorDistances.COSINE
   ),
    generative_config=Configure.Generative.openai()

)


In [ ]:
#
# View Collections
#

client.collections.list_all()

### IMPORT DATA: Weaviate Data Import

Now that we have created our collections we can populate each collection with out dataset data.

**Dynamic Batch Import:**
To populate the 'Game' collection we use the dynamic batch import approach.

**Insert Many Import:**
To populate the 'Player' collection we use the insert many import approach.


**Resources:**
- Weaviate Batch Import: [Documentation](https://weaviate.io/developers/weaviate/manage-data/import)



In [7]:
#
# Access Collections: Player and Game
#

game = client.collections.get("Game")
player = client.collections.get("Player")

In [ ]:
#
# Batch Import Game data from .csv
#

with game.batch.dynamic() as batch:
    for index, row in data.iterrows():
        data_row = {
            "title": row['Name'],
            "subtitle": row['Subtitle'],
            "primaryType": row['Type_1'],
            "secondaryType": row['Type_2'],
            "description": row['wiki_game_summary'],
            "minPlayers": row['MinPlayers'],
            "maxPlayers": row['MaxPlayers'],
            "bestPlayers": row['BestPlayers'],
            "minPlayTime": row['MinPlayTime'],
            "minAge": row['MinAge']
        }
        try:
          obj_uuid = generate_uuid5(data_row)
          batch.add_object(
              properties=data_row,
              uuid=obj_uuid
          )
        except Exception as e:
          print(f"Error occurred for title '{row['Name']}' at index {index}: {e}")

In [ ]:
#
# Check for failed imports
#

len(game.batch.failed_objects)

In [ ]:
#
# Populate Player Collection : insert_many()
#

player = client.collections.get("Player")
player.data.insert_many(player_profiles_json)
len(player.batch.failed_objects)

### Generative Search: Querying the Collections



In [ ]:
#
# Test Fetch Query for Player and Game collection
#

game_response = game.query.fetch_objects(
    limit=1
)

player_response = player.query.fetch_objects(
    limit=1
)

print(f'Game Fetch: {game_response}')
print(f'Player Fetch: {player_response}')

In [12]:
#
# Query
#

response = game.query.near_text(
    query="fantasy adventure card game",
    limit=2,
    return_metadata=MetadataQuery(distance=True)
)

print_response_objects(response.objects)

Properties: {'bestPlayers': 2.0, 'description': 'The Lord of the Rings: The Card Game is a non-collectible customizable card game produced by Fantasy Flight Games. As part of the Living Card Game (LCG) genre, it is a cooperative and strategic card game set in Middle-earth, a fantasy world featured in literary works by J. R. R. Tolkien, including The Hobbit and The Lord of the Rings. Its digital adaptation, titled The Lord of the Rings: Adventure Card Game, is published by Asmodee Digital for cross-platform play on Microsoft Windows, macOS, PlayStation 4, Xbox One and Nintendo Switch, being the "first ever digital LCG".\n\n\n== Mechanics ==\nLike other card games in Fantasy Flight\'s "Living Card Game" line, it has deck construction and regularly released expansions similar to a collectible card game, but without the randomized card distribution.\nThe game may be played either solitaire or with a group. Up to 2 players can play the game with a single core set, and up to 4 players can pl

In [9]:
#
# Generator
#
client.connect()
generate_prompt = "Tell me why this is a suitable game based on what I am looking for, and why this game is so much fun, include a brief summary of the game"

response = game.generate.near_text(
    query="fantasy adventure card game",
    grouped_task=generate_prompt,
    limit=1
)

print(f'Generated output: {response.generated}')
print_response_objects(response.objects)


Generated output: The Lord of the Rings: Journeys in Middle-Earth is a suitable game for you if you are looking for a cooperative and strategic card game set in the fantasy world of Middle-earth. This game allows players to work together against the forces of Sauron, rather than competing against each other, which can be a refreshing change of pace. The deck construction aspect of the game adds depth and customization, allowing players to create their own unique strategies.

The game is a lot of fun because it immerses players in the rich world of Middle-earth, with thematic content, artwork, and challenges that are replayable in different ways. The cooperative gameplay and player interaction make it engaging and exciting to explore the epic campaign. The digital version of the game also brings a new PvE and cooperative gameplay experience to the digital collectible card game industry.

In summary, The Lord of the Rings: Journeys in Middle-Earth is a thrilling and immersive game where 

In [ ]:
print(response.objects.)

In [51]:
#
# Query Player Collection : pass game abjoct as query and return the player(s with the greatest similarity to the game)
#



test_game_profile = {'bestPlayers': 2.0, 'maxPlayers': 5.0, 'description': 'The Lord of the Rings: The Card Game is a non-collectible customizable card game produced by Fantasy Flight Games. As part of the Living Card Game (LCG) genre, it is a cooperative and strategic card game set in Middle-earth, a fantasy world featured in literary works by J. R. R. Tolkien, including The Hobbit and The Lord of the Rings. Its digital adaptation, titled The Lord of the Rings: Adventure Card Game, is published by Asmodee Digital for cross-platform play on Microsoft Windows, macOS, PlayStation 4, Xbox One and Nintendo Switch, being the "first ever digital LCG".\n\n\n== Mechanics ==\nLike other card games in Fantasy Flight\'s "Living Card Game" line, it has deck construction and regularly released expansions similar to a collectible card game, but without the randomized card distribution.\nThe game may be played either solitaire or with a group. Up to 2 players can play the game with a single core set, and up to 4 players can play with an additional core set. Unlike most card games of this type, this game is a cooperative game; the players are not opponents, but work together against an "encounter deck" which represents the forces of Sauron and produces the obstacles that the players have to conquer.\nEach player has their own cards: their heroes (up to 3) which start in play, and a deck of at least 50 cards, composed of Allies, Attachments (such as weapons, armor, and other items), and Events. Although the core set comes with pre-constructed decks, deck construction is a major aspect of the game, and many players build their own decks, utilising cards from the various expansions.\n\n\n== Community ==\nThe Lord of the Rings Living Card Game has an active following, with multiple podcasts and blogs dedicated to discussing gameplay, strategy, deck-building, and news, as well as providing custom content. Podcasts include Cardboard of the Rings, The Grey Company, and The Mouth of Sauron. Blogs include Tales From the Cards, Hall of Beorn, Master of Lore, Dor Cuarthol, and Vision of the Palantir.\n\n\n== Expansions ==\nThere are several types of expansion to this game. Each Deluxe Expansion (and the core set) contain 3 quests and begins a new cycle, which usually shares a mechanical or story-driven theme. These releases are supplemented by 6 Adventure Packs for a total of 9 quests per cycle. The Hero and Campaign Expansions combine a cycle\'s Deluxe Expansion and its related Adventure packs (i.e. The Lost Realm expansion and the six Angmar Awakened packs) into two products, while also adding campaign cards. The Saga Expansions follow the story of The Hobbit and The Lord of the Rings, with the initial time-frame for the game the 17 years between Bilbo leaving The Shire and Frodo following him. The latter introduced a mode of play called Campaign mode allowing for some degree of permanence between the quests telling the story. Players can earn powerful "Boon" cards, which can be used in the following quests, as well as "Burden" cards which make future quests more difficult.\nThe Standalone Scenarios are one-off quests released without any playing cards. Fantasy Flight Games usually releases one to coincide with their Gen Con event and their Fellowship (in the house) event. They have often been more difficult than a standard quest and allow the designers to experiment with ideas outside the confines of the main game.\n\nAdventure Pack Cycles continue the story of Deluxe Expansions, or, in the case of the Shadows of Mirkwood Cycle, the Base Game. Each Deluxe expansion provides encounter cards for three scenarios, and each Adventure Pack in the associated Cycle provides encounter cards for one more scenario. Both types of expansions also introduce new player cards. An Adventure pack consists of 60 fixed cards.\n\n\n== Adventure Card Game ==\nIn October 2017, Asmodee Digital and Fantasy Flight announced the formation of Fantasy Flight Interactive, a division of the merged companies to bring more of Fantasy Flight\'s physical board games to digital implementations. Among the games, developed by the studio was The Lord of the Rings: Adventure Card Game. Fantasy Flight Interactive released it on 28 August 2018 with the title The Lord of the Rings Living Card Game in an early access phase. It was fully released on 29 August 2019 on PC and Mac (Steam), and by the end of the year on PlayStation 4, Xbox One, and Nintendo Switch (developed by Virtuos) with cross-platform play. In December 2019 (Steam) and early 2020 (other platforms) it was upgraded under the title The Lord of the Rings: Adventure Card Game - Definitive Edition which includes all additional content as well as a new mode. However, as part of company-wide layoffs, the Fantasy Flight Interactive division was closed down in January 2020. Following the closure of Fantasy Flight Interactive, by February 2020, Asmodee announced that it was opening up its library of board games to be made into digital versions through licensing options to any developer. Since mid-2020, the game\'s developing was continued by Antihero Studios, who made a major update in October 2020 introducing offline play mode and new Adventure mode with free pack "The Fords of Isen" among others.\nThe game is not a direct adaptation but heavily inspired by the original card game. It has campaign stories, adventures, and quests that are narrated and with different paths, characters who are voiced, and dynamic reactions by the artificial intelligence of Sauron (and Saruman) in three difficulty levels (standard, advanced, and expert). It is based on player versus environment experience (PvE) intended for a solo player and online co-op. Decks are composed of 3 heroes and 30 common cards out of many heroes and different card types (allies, attachments, and events), with all the cards divided into four categories (purple "leadership", green "lore", blue "spirit", and red "tactics"), which the player can combine in deck building making various decks besides the pre-built decks with different styles of play. The cards have a cost value to play, with characters having an attack, health, and willpower number, with the latter used for completing objectives or raising the player\'s meter to activate bonus abilities. Sauron also has his own cards and "threat" meter which uses to activate bonus events and when full makes an automatic defeat of the player. Each round, the player and the enemy take turns playing a single action until all playable options are exhausted.\n\n\n== Reception ==\nBoth versions were released to positive reception. The 2018 Zatu Games review gave a score of 85 out of 100 to the tabletop version, praising the quality of the thematic content, artwork, game structure, challenges that are replayable by different approaches, and player interaction, making it especially attractive to those wanting to explore the world of Middle-earth. However both with review by HCL, noted it\'s needed to keep in mind it has a demanding learning curve to successfully beat the game and the number of expansions. It received several nominations at the Golden Geek Awards for the best card game, thematic board game, board game artwork/presentation, and 2-player board game category.\nThe digital version on review aggregator website Metacritic has a score of 70-75 out of 100 based on few reviews published before the release of the definitive edition in December 2019. Although still not as expansive in content as the tabletop version, they praised that the game is bringing a new PvE and cooperative gameplay to the digital collectible card game industry, the art, story-telling, voice acting, and game mechanic which cards are a bit simplified in comparison to tabletop due to the format, having resemblance to Hearthstone, but none the less complexity and strategy to tabletop. Chris Carter writing for Destructoid found it "one of the most enjoyable games I\'ve ever experienced, tabletop or otherwise" and concluded that the way Living Card Game (LCG) genre is created and developed through the game "could be a guiding light in an increasingly predatory industry. With loot box legislation around the corner ... it may launch at a perfect time".\n\n\n== See also ==\nMiddle-earth Collectible Card Game (out-of-print)\nThe Lord of the Rings Trading Card Game (out-of-print)\n\n\n== References ==\n\n\n== External links ==\nThe Card Game Official Website\nAdventure Card Game Official Website\nLord of the Rings: The Card Game   at BoardGameGeek', 'minAge': 14.0, 'minPlayTime': 60.0, 'title': 'The Lord of the Rings: Journeys in Middle-Earth', 'subtitle': 'Heroes explore Middle-earth, working together to combat evil in an epic campaign.', 'minPlayers': 1.0, 'secondaryType': 'NaN', 'primaryType': 'Thematic'}

response = player.query.near_text(
    query=f"{test_game_profile}",
    limit=2,
    return_metadata=MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.distance)

{'age': 28.0, 'name': 'Regine', 'profile': 'Catan is my go-to game for its blend of strategy and social interaction. I relish the challenge of building settlements and trading resources with fellow players. Cooperative games that foster camaraderie are my favorite.', 'favoriteGame': 'Catan'}
0.24472182989120483
{'age': 22.0, 'favoriteGame': 'Risk', 'profile': 'Risk appeals to my adventurous spirit and strategic thinking. I enjoy plotting my conquests and engaging in epic battles with friends. Games that require long-term planning and diplomacy are my forte.', 'name': 'Sinclair'}
0.24544823169708252


In [ ]:
#
# Close client connection
#

client.close()

In [ ]:
#

#Continue on to: Part Two


In the next section we will use Open AI function calling to process user input, and demo our workflow...

